In [1]:
import pandas as pd  # 数据分析库
from snownlp import SnowNLP  # 中文情感分析库
from wordcloud import WordCloud  # 绘制词云图
from pprint import pprint  # 美观打印
import jieba.analyse  # jieba分词
from PIL import Image  # 读取图片
import numpy as np  # 将图片的像素点转换成矩阵数据

In [2]:
# 情感分析打标
def sentiment_analyse(v_cmt_list):
    """
    情感分析打分
    :param v_cmt_list: 需要处理的评论列表
    :return:
    """
    score_list = []  # 情感评分值
    tag_list = []  # 打标分类结果
    pos_count = 0  # 计数器-积极
    neg_count = 0  # 计数器-消极
    for comment in v_cmt_list:
        tag = ''
        sentiments_score = SnowNLP(comment).sentiments
        if sentiments_score < 0.3:
            tag = '消极'
            neg_count += 1
        else:
            tag = '积极'
            pos_count += 1
        score_list.append(sentiments_score)  # 得分值
        tag_list.append(tag)  # 判定结果
    print('积极评价占比：', round(pos_count / (pos_count + neg_count), 4))
    print('消极评价占比：', round(neg_count / (pos_count + neg_count), 4))
    df['情感得分'] = score_list
    df['分析结果'] = tag_list
    # 把情感分析结果保存到excel文件
    df.to_excel('谷爱凌_情感评分结果.xlsx', index=None)
    print('情感分析结果已生成：谷爱凌_情感评分结果.xlsx')

In [3]:
def make_wordcloud(v_str, v_stopwords, v_outfile):
    """
    绘制词云图
    :param v_str: 输入字符串
    :param v_stopwords: 停用词
    :param v_outfile: 输出文件
    :return: None
    """
    print('开始生成词云图：{}'.format(v_outfile))
    try:
        stopwords = v_stopwords  # 停用词
        backgroud_Image = np.array(Image.open('谷爱凌背景图.png'))  # 读取背景图片
        wc = WordCloud(
            background_color="white",  # 背景颜色
            width=1500,  # 图宽
            height=1200,  # 图高
            max_words=1000,  # 最多字数
            # 字体文件路径，根据实际情况(Mac)替换
            font_path='/System/Library/Fonts/SimHei.ttf',
            # font_path="C:\Windows\Fonts\simhei.ttf",  # 字体文件路径，根据实际情况(Windows)替换
            stopwords=stopwords,  # 停用词
            mask=backgroud_Image,  # 背景图片
        )
        jieba_text = " ".join(jieba.lcut(v_str))  # jieba分词
        wc.generate_from_text(jieba_text)  # 生成词云图
        wc.to_file(v_outfile)  # 保存图片文件
        print('词云文件保存成功：{}'.format(v_outfile))
    except Exception as e:
        print('make_wordcloud except: {}'.format(str(e)))

In [4]:
df = pd.read_csv('谷爱凌弹幕.csv')  # 读取excel

In [6]:
v_cmt_list = df['弹幕内容'].values.tolist()  # 评论内容列表
print('length of v_cmt_list is:{}'.format(len(v_cmt_list)))

length of v_cmt_list is:2661


In [7]:
v_cmt_list = [str(i) for i in v_cmt_list]  # 数据清洗-list所有元素转换成字符串

In [8]:
v_cmt_str = ' '.join(str(i) for i in v_cmt_list)  # 评论内容转换为字符串

In [9]:
# 1、情感分析打分
sentiment_analyse(v_cmt_list=v_cmt_list)

积极评价占比： 0.9049
消极评价占比： 0.0951
情感分析结果已生成：谷爱凌_情感评分结果.xlsx


In [11]:
# 2、用jieba统计弹幕中的top10高频词
keywords_top10 = jieba.analyse.extract_tags(v_cmt_str, withWeight=True, topK=10)
print('top10关键词及权重：')
pprint(keywords_top10)

top10关键词及权重：
[('加油', 0.5639019615961216),
 ('祝福', 0.2097939445347145),
 ('谷爱凌', 0.14626550778425776),
 ('卧槽', 0.12094118674932029),
 ('中国', 0.06708173897846384),
 ('键盘', 0.0659939099894372),
 ('厉害', 0.060244303047569336),
 ('国籍', 0.059496894882816745),
 ('天才', 0.05499776168548123),
 ('妈妈', 0.04990450209996737)]


In [12]:
# 3、画词云图
make_wordcloud(v_str=v_cmt_str,
               v_stopwords=['的', '啊', '她', '是', '了', '你', '我', '都',
                            '也', '不', '在', '吧', '说', '就是', '这', '有'],  # 停用词
               v_outfile='谷爱凌弹幕_词云图.jpg'  # 词云图文件名
               )

开始生成词云图：谷爱凌弹幕_词云图.jpg
词云文件保存成功：谷爱凌弹幕_词云图.jpg
